# Módulo 5: Clasificación taxónomica de las secuencias

## Descripción general

Kraken es un clasificador de secuencias taxonómicas que asigna etiquetas taxonómicas a lecturas cortas de ADN. Lo hace examinando los k-mers dentro de una lectura y consultando una base de datos con esos k-mers. Esta base de datos contiene un mapeo de cada k-mer en la librería genómica de Kraken hasta el ancestro común más bajo (LCA) en un árbol taxonómico de todos los genomas que contienen ese k-mer. El conjunto de taxones LCA que corresponden a los k-mers en una lectura luego se analizan para crear una etiqueta taxonómica única para la lectura; esta etiqueta puede ser cualquiera de los nodos del árbol taxonómico. Kraken está diseñado para ser rápido, sensible y muy preciso. Nuestras pruebas en varios datos reales y simulados han demostrado que Kraken tiene una sensibilidad ligeramente inferior a Megablast con una precisión ligeramente superior. En un conjunto de lecturas simuladas de 100 pb, Kraken procesó más de 1,3 millones de lecturas por minuto en un solo núcleo en funcionamiento normal y más de 4,1 millones de lecturas por minuto en funcionamiento rápido.

![kraken](images/kraken.png)

*Tomado de https://link.springer.com/article/10.1186/gb-2014-15-3-r46*

Otras lecturas: https://github.com/DerrickWood/kraken2/wiki/Manual




### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Instalar kraken2
!conda install bioconda::kraken2

In [ ]:
# Verificar si Kraken2 está instalado
!kraken2 -h

In [ ]:
#Descargue SRA tools usando bioconda
!conda install -c bioconda sra-tools

### Descargar datos

In [ ]:
!wget https://zenodo.org/records/13178194/files/Module_5.tar.gz

### Extraer el archivo .tar.gz

In [ ]:
!tar xvf Module_5.tar.gz

Navegaremos a la carpeta que contiene los archivos pareados FASTQ ERR2667698_1.fastq.gz y ERR2667698_2.fastq.gz los cuales corresponden al run accession ERR2667698 del proyecto [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084)	

Algunos datos importantes de la muestra:

- País de origen: Perú
- Organismo: *Streptococcus pneumoniae*
- Instrumento: ILLUMINA
- Modelo del instrumento: Illumina MiSeq 
- Conteo de reads: 2445004
- Conteo de bases: 369195604 
- Nombre del centro: Wellcome Sanger Institute;SC 
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS

In [ ]:
%cd Module_5

## Clasificación

Usaremos Kraken2 en esta sección. Para clasificar un conjunto de secuencias (lecturas), usaremos el comando:

In [ ]:
# Correr kraken2
!kraken2 --use-names --db Kraken/k2_standard_08gb_20221209  --paired ERR2667698_1.fastq.gz ERR2667698_2.fastq.gz --report kraken_report.txt --output -

Una explicación de este comando es la siguiente:

**kraken2**: es la herramienta

**--use-names**:  imprime nombres científicos en lugar de solo taxids

**--db** Kraken/k2_standard_08gb_20221209: Ruta y nombre para la base de datos de Kraken 2

**--paired** ERR2667698_1.fastq.gz ERR2667698_2.fastq.gz: proporciona lecturas emparejadas

**--report** kraken_report.txt: genera un informe con recuentos agregados/clado para archivar

**--output -**: Suprimir salida para salida estándar

Puede verificar la salida de kraken usando `!head kraken_report.txt`

In [ ]:
# Consulte el informe

Deberías obtener:

- **1ra columna**: Porcentaje de fragmentos cubiertos por el clado enraizado en este taxón. 

- **2da columna**: Número de fragmentos cubiertos por el clado enraizado en este taxón 

- **3ra columna**: Número de fragmentos asignados directamente a este taxón 

- **4ta columna**: Un código de rango, que indica (S)no clasificado, (R)oot, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rden, (F)amily, (G) )enus, o (S)especies. Los taxones que no están en ninguno de estos 10 rangos tienen un código de rango que se forma usando el código de rango del rango de ancestro más cercano con un número que indica la distancia desde ese rango. Por ejemplo, "G2" es un código de rango que indica que un taxón se encuentra entre el género y la especie y el taxón abuelo se encuentra en el rango de género. 

- **5ta columna**: número de identificación taxonómica del NCBI 

- **6ta columna**: nombre científico con sangría


Para encontrar el porcentaje de fragmentos cubiertos por *Streptoccocus pneumoniae*, ejecute:

In [ ]:
!grep "Streptococcus pneumoniae" kraken_report.txt

Debería obtener:

## Control de calidad de taxonomía

Ahora calcularemos el porcentaje de lecturas de *Streptococcus pneumoniae* y en base a eso evaluaremos si la muestra pasa o no pasa.

El límite de corte para *Streptococcus pneumoniae* es 60% ≤60% = FALLO 60% = APROBADO

La muestra debe tener más del 60% de las lecturas de Streptococcus pneumoniae.

Ejecutaremos un script bash personalizado para realizar la evaluación.

In [ ]:
!bash percentage_calculator.sh

## BONO!

Si está trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puede optimizar los comandos, los bucles (for loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Cree un nuevo script bash usando nano llamado `taxonomy.sh`

In [ ]:
# No ejecute esta celda
# Crear un script para la taxonomía
!nano taxonomy.sh

Luego copie y pegue el siguiente scrip en su nuevo archivo:

In [ ]:
#!/bin/bash
#Author: Nathalia Portilla

for i in $(ls *_1.trimmed.fastq.gz); do

NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"

kraken2 --use-names --db k2_standard_08gb_20221209 --paired $j $k --report ${NAME}.kraken.txt --output -
done

Guarde el archivo.

Finalmente, puede ejecutarlo:

In [ ]:
# No ejecute esta celda
!bash taxonomy.sh